# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [19]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [20]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

#df_densidad_alta = pd.read_csv("instancias/densidad-alta/indice.csv");
#df_densidad_baja = pd.read_csv("instancias/densidad-baja/indice.csv");
df_mejor_caso_bt = pd.read_csv("instancias/mejor-caso-bt/indice.csv");
#df_peor_caso_bt = pd.read_csv("instancias/peor-caso-bt/indice.csv");
#df_dinamica = pd.read_csv("instancias/dinamica/indice.csv");

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- DP: Programación dinámica.

In [21]:
# Devuelve el tiempo de ejecucion
def correr_experimento(metodo, archivo_instancia): #archivo_instancia es el PATH al test.
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../social_distancing", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar. OKEY. ACA. 
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [22]:
experimentos = [];

### Experimento 1
Correr Fuerza Bruta en las primeras 30 instancias de densidad-alta y densidad-baja.

Mio: Correr FB de la instancia 30 a la 60 de ambas densidades.

In [11]:
for n in range(10, 35):
    fila_n = df_densidad_alta.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "FB", fila_n["archivo"]]);

for n in range(10, 35):
    fila_n = df_densidad_baja.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "FB", fila_n["archivo"]]);

## Experimento 2
Correr BT para mejor y peor caso.

In [23]:
for n in range(0, df_mejor_caso_bt.shape[0]):
    fila_n = df_mejor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);

#for n in range(0, df_peor_caso_bt.shape[0]):
 #   fila_n = df_peor_caso_bt.iloc[n];
  #  experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);

## Experimento 3
Correr BT, BT-F, BT-O para densidad alta y baja.

In [ ]:
for n in range(0, df_densidad_alta.shape[0]):
    fila_n = df_densidad_alta.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-O", fila_n["archivo"]]);

for n in range(0, 30):
    fila_n = df_densidad_baja.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-O", fila_n["archivo"]]);

# Experimento 4
Correr DP para el dataset _dinamica_.

In [22]:
for i in range(0, df_dinamica.shape[0]):
    fila = df_dinamica.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["W"], "DP", fila["archivo"]]);

## Experimento 5
Correr DP para el dataset densidad-alta y densidad-baja.

In [ ]:
for i in range(0, df_densidad_alta.shape[0]):
    fila = df_densidad_alta.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["W"], "DP", fila["archivo"]]);

#for i in range(0, df_densidad_baja.shape[0]):
  #  fila = df_densidad_baja.iloc[i];
   # experimentos.append([fila["dataset"], fila["n"], fila["W"], "DP", fila["archivo"]]);

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [24]:
columnas = ["dataset", "n", "W", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
#print(experimentos)
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        #print(experimento[3])
        tiempos.append(correr_experimento(experimento[3], experimento[4]));
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);
    print(filas)
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 499/499'

[['mejor-caso-bt', 0, 1000, 'BT', 0.0017], ['mejor-caso-bt', 1, 1000, 'BT', 0.0021], ['mejor-caso-bt', 2, 1000, 'BT', 0.0024], ['mejor-caso-bt', 3, 1000, 'BT', 0.0027], ['mejor-caso-bt', 4, 1000, 'BT', 0.003], ['mejor-caso-bt', 5, 1000, 'BT', 0.0032], ['mejor-caso-bt', 6, 1000, 'BT', 0.0035], ['mejor-caso-bt', 7, 1000, 'BT', 0.0055], ['mejor-caso-bt', 8, 1000, 'BT', 0.0041], ['mejor-caso-bt', 9, 1000, 'BT', 0.0043], ['mejor-caso-bt', 10, 1000, 'BT', 0.0045], ['mejor-caso-bt', 11, 1000, 'BT', 0.0046], ['mejor-caso-bt', 12, 1000, 'BT', 0.0052], ['mejor-caso-bt', 13, 1000, 'BT', 0.0054], ['mejor-caso-bt', 14, 1000, 'BT', 0.0058], ['mejor-caso-bt', 15, 1000, 'BT', 0.0067], ['mejor-caso-bt', 16, 1000, 'BT', 0.0062], ['mejor-caso-bt', 17, 1000, 'BT', 0.0068], ['mejor-caso-bt', 18, 1000, 'BT', 0.0069], ['mejor-caso-bt', 19, 1000, 'BT', 0.0073], ['mejor-caso-bt', 20, 1000, 'BT', 0.0079], ['mejor-caso-bt', 21, 1000, 'BT', 0.0076], ['mejor-caso-bt', 22, 1000, 'BT', 0.008], ['mejor-caso-bt', 23, 

In [13]:
print("Hi")

Hi
